In [ ]:
import os

In [ ]:
!git clone https://github.com/terryluan12/Genrify.git

In [ ]:
!cd Genrify && make

In [ ]:
import sys
sys.path.insert(0, "Genrify/src")

In [ ]:
from datasources import download_datasets
from preprocessing import preprocess

In [ ]:
if not os.path.isdir("Genrify/src/datasources/processed_data"):
    download_datasets("Genrify/src")

preprocess("Genrify/src")